In [22]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from tqdm import tqdm_notebook as tqdm

import json
import random
import re
import os
import sys
import math
import pickle
import csv

import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer  
from nltk.stem import SnowballStemmer 
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from collections import Counter
from string import digits
from collections import Counter



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cengqiqi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/cengqiqi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/cengqiqi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [23]:
# load docs
with open('inverted_word_dictionary.txt', 'rb') as handle:
    inverted_word_dictionary = pickle.loads(handle.read())
    
with open("doc_length_list.txt", "rb") as fp:   # Unpickling
    doc_length_list = pickle.load(fp)
    
# load doc
path_windows = "N:\\DesktopSettings\\Desktop\\DM_working\\dataset\\wiki_id_text"
path_mac = "/Users/cengqiqi/Desktop/DM_working/dataset/wiki_id_text"
dataset_wikipage = pd.read_table(path_mac,header = None)
wikipage = dataset_wikipage

In [24]:
# load claims
def load_dataset_json(path, instance_num=1e6):
    """
    Reads the Fever Training set, returns list of examples.
    instance_num: how many examples to load. Useful for debugging.
    """
    data = []
    with open(path, 'r') as openfile:
        for iline, line in enumerate(openfile.readlines()):
            data.append(json.loads(line))
            if iline+1 >= instance_num:
                break
    return data
path_windows = "N:\\DesktopSettings\\Desktop\\DM_working\\dataset\\train.jsonl"
path_mac = "/Users/cengqiqi/Desktop/DM_working/dataset/train.jsonl"
dataset = load_dataset_json(path=path_mac, instance_num=20)

# get first 10 verifiable claims
# [75397, 150448, 214861, 156709, 129629, 33078, 6744, 226034, 40190, 76253].

claims_10 = []
for i in dataset:
    if i['verifiable'] == 'VERIFIABLE':
        claims_10.append(i)
claims_10 = claims_10[0:10]

<font face="微软雅黑" size=6 color=#000000> query-likelihood unigram language model

In [34]:
def claimProcess(words):
    #######  lower case  ############################################################
    words = words.lower()
    
    ####### tokenize ###############################################################
    pattern = r"""(?x)                  
                          (?:[A-Z]\.)+           # abbreviations, e.g. U.S.A. 
                          |\$?\d+(?:,\d+)*(?:\.\d+)?%? # 2,000 or 2.5
                          |\w+(?:[-']\w+)*      # words w/ optional internal hyphens/apostrophe  e.g. can't
                        """
    word_list = nltk.regexp_tokenize(words, pattern)
    
    ####### remove stop words #######################################################
    stopwordlist = set(stopwords.words('english'))
    word_list = [w for w in word_list if w not in stopwordlist]
   
    ####### lemmatize ################################################################
    word_list = [WordNetLemmatizer().lemmatize(w,pos = 'v') for w in word_list] # only Lancaster
    

    return word_list 
    

In [35]:
def queryLikelihood(claim_wordlist):
    
    p_for_doc_list = []
    for i in range(len(doc_length_list)):
        if doc_length_list[i]:
            p = 1
            for word_c in claim_wordlist:
                word_count_in_doc = inverted_word_dictionary[word_c]["doc" + str(i)] if "doc" + str(i) in inverted_word_dictionary[word_c] else 0
                p = p*(word_count_in_doc/doc_length_list[i])
        else:
            p = 0
        p_for_doc_list.append(p)
        
    return p_for_doc_list

# claim_wordlist = claimProcess(claims_10[0]['claim'])
# p_for_doc_list = queryLikelihood(claim_wordlist)

In [36]:
# # get the top 5 doc id for a single claim
def getTop5(p_for_doc_list):

    top_5_idx = np.argsort(p_for_doc_list)[-5:][::-1]
    
    return top_5_idx

In [37]:
# # find the five most similar documents for ten claim and write them to csv
# for i in range(len(claims_10)):
    
#     claim_wordlist = claimProcess(claims_10[i]['claim'])
#     p_for_doc_list = queryLikelihood(claim_wordlist)
#     doc_id = getTop5(p_for_doc_list)

#     with open('q3_non_smoothing.csv', "a", encoding="utf-8", newline='') as config_csv:
#         writer = csv.writer(config_csv)
#         writer.writerow([f"The five most similar documents for Claim {claims_10[i]['id']} \n"])
#         for top_id in doc_id:
#             writer.writerow([str(top_id), str(wikipage[0][top_id])])
#         writer.writerow(f"\n")

In [38]:
# find the five most similar documents for ten claim and write them to csv
    
with open('q3.csv', "a", newline='') as config_csv:
    list_save = ['non_smoothing: ']
    writer = csv.writer(config_csv)
    writer.writerow(list_save)
    
    list_save = ['claim id', 'doc id_1', 'doc id_2', 'doc id_3', 'doc id_4', 'doc id_5']
    writer = csv.writer(config_csv)
    writer.writerow(list_save)
    
for i in tqdm(range(0,len(claims_10))):
    
    claim_wordlist = claimProcess(claims_10[i]['claim'])
    p_for_doc_list = queryLikelihood(claim_wordlist)
    doc_id = getTop5(p_for_doc_list)

    with open('q3.csv', "a", newline='') as config_csv:
        
        list_save = [claims_10[i]['id']]
        list_save.extend(wikipage[0][doc_id])
        
        writer = csv.writer(config_csv)
        writer.writerow(list_save)

<font face="微软雅黑" size=6 color=#000000>  Laplace Smoothing

In [39]:
# count the number of 
# a = np.array(doc_words_list)
# v = len(np.unique(a))
v = len(inverted_word_dictionary)

def laplaceSmoothing(claim_wordlist):
    
    p_for_doc_list = []
    for i in range(len(doc_length_list)):
        logp = 0
        for word_c in claim_wordlist: 
            word_count_in_doc = inverted_word_dictionary[word_c]["doc" + str(i)] if "doc" + str(i) in inverted_word_dictionary[word_c] else 0
            temp = (1+word_count_in_doc)/(v+doc_length_list[i]) 
            logp = logp+math.log(temp) # use log to avoid overflow
      
        p_for_doc_list.append(math.pow(math.e,logp))
        
                
    return p_for_doc_list

# claim_wordlist = claimProcess(claims_10[0]['claim'])
# p_for_doc_list = laplaceSmoothing(claim_wordlist)
# getTop5(p_for_doc_list)

In [40]:
# for i in range(len(claims_10)):
    
#     claim_wordlist = claimProcess(claims_10[i]['claim'])
#     p_for_doc_list = laplaceSmoothing(claim_wordlist)
#     doc_id = getTop5(p_for_doc_list)

#     with open('q3_laplace_smoothing.csv', "a", newline='') as config_csv:
#         writer = csv.writer(config_csv)
#         writer.writerow([f"The five most similar documents for Claim {claims_10[i]['id']} \n"])
#         for top_id in doc_id:
#             writer.writerow([str(top_id), str(wikipage[0][top_id])])
#         writer.writerow(f"\n")

In [41]:
# find the five most similar documents for ten claim and write them to csv
    
with open('q3.csv', "a", newline='') as config_csv:

    list_save = [' ']
    writer = csv.writer(config_csv)
    writer.writerow(list_save)
    
    list_save = ['laplace_smoothing: ']
    writer = csv.writer(config_csv)
    writer.writerow(list_save)
    
    list_save = ['claim id', 'doc id_1', 'doc id_2', 'doc id_3', 'doc id_4', 'doc id_5'] 
    writer = csv.writer(config_csv)
    writer.writerow(list_save)
    
for i in tqdm(range(0,len(claims_10))):
    
    claim_wordlist = claimProcess(claims_10[i]['claim'])
    p_for_doc_list = laplaceSmoothing(claim_wordlist)
    doc_id = getTop5(p_for_doc_list)

    with open('q3.csv', "a", newline='') as config_csv:
        
        list_save = [claims_10[i]['id']]
        list_save.extend(wikipage[0][doc_id])
        
        writer = csv.writer(config_csv)
        writer.writerow(list_save)

<font face="微软雅黑" size=6 color=#000000>  Jelinek-Mercer Smoothing

In [42]:
# total_words = 0
# for i in doc_words_list:
#     total_words += len(i)

# total_words = 0
# for word_c in tqdm(inverted_word_dictionary):
#         total_words += np.sum(list(inverted_word_dictionary[word_c].values()))
        
total_words = 289459793

In [43]:
def JMSmoothing(claim_wordlist):
    
    word_count_in_collection = []
    for word_c in claim_wordlist:
        word_count_in_collection.append(np.sum(list(inverted_word_dictionary[word_c].values())))
        
    p_for_doc_list = []
    for i in range(len(doc_length_list)):
        logp = 0
        for j in range(len(claim_wordlist)): 
            
            if doc_length_list[i]:
                word_count_in_doc = inverted_word_dictionary[claim_wordlist[j]]["doc" + str(i)] if "doc" + str(i) in inverted_word_dictionary[claim_wordlist[j]] else 0
                temp = 0.5*(word_count_in_doc/doc_length_list[i]) + 0.5*(word_count_in_collection[j]/total_words)
            else:
                temp = 0.5*(word_count_in_collection[j]/total_words)
                
            logp = logp+math.log(temp)
      
        p_for_doc_list.append(math.pow(math.e,logp))
        
                
    return p_for_doc_list

# claim_wordlist = claimProcess(claims_10[1]['claim'])
# p_for_doc_list = laplaceSmoothing(claim_wordlist)
# getTop5(p_for_doc_list)

In [44]:
# for i in range(len(claims_10)):
    
#     claim_wordlist = claimProcess(claims_10[i]['claim'])
#     p_for_doc_list = JMSmoothing(claim_wordlist)
#     doc_id = getTop5(p_for_doc_list)

#     with open('q3_JM_smoothing.csv', "a", newline='') as config_csv:
#         writer = csv.writer(config_csv)
#         writer.writerow([f"The five most similar documents for Claim {claims_10[i]['id']} \n"])
#         for top_id in doc_id:
#             writer.writerow([str(top_id), str(wikipage[0][top_id])])
#         writer.writerow(f"\n")

In [45]:
# find the five most similar documents for ten claim and write them to csv
    
with open('q3.csv', "a", newline='') as config_csv:
    
    list_save = [' ']
    writer = csv.writer(config_csv)
    writer.writerow(list_save)
    
    list_save = ['JM_smoothing: ']
    writer = csv.writer(config_csv)
    writer.writerow(list_save)
    
    list_save = ['claim id', 'doc id_1', 'doc id_2', 'doc id_3', 'doc id_4', 'doc id_5'] 
    writer = csv.writer(config_csv)
    writer.writerow(list_save)
    
for i in tqdm(range(0,len(claims_10))):  
    
    claim_wordlist = claimProcess(claims_10[i]['claim'])
    p_for_doc_list = JMSmoothing(claim_wordlist)
    doc_id = getTop5(p_for_doc_list)

    with open('q3.csv', "a", newline='') as config_csv:
        
        list_save = [claims_10[i]['id']]
        list_save.extend(wikipage[0][doc_id])
        
        writer = csv.writer(config_csv)
        writer.writerow(list_save)

<font face="微软雅黑" size=6 color=#000000>  Dirichlet Smoothing

In [46]:
total_words = 289459793
#ave_length = math.ceil(total_words/len(doc_length_list))
ave_length = 54

In [47]:
def DirichletSmoothing(claim_wordlist):
    
    word_count_in_collection = []
    for word_c in claim_wordlist:
        word_count_in_collection.append(np.sum(list(inverted_word_dictionary[word_c].values())))
           
    p_for_doc_list = []
    for i in range(len(doc_length_list)):
        logp = 0
        for j in range(len(claim_wordlist)): 
            if doc_length_list[i]:
                word_count_in_doc = inverted_word_dictionary[claim_wordlist[j]]["doc" + str(i)] if "doc" + str(i) in inverted_word_dictionary[claim_wordlist[j]] else 0
                temp1 = (doc_length_list[i]/(doc_length_list[i]+ave_length))*(word_count_in_doc/doc_length_list[i])
            else:
                temp1 = 0
            temp2 = (ave_length/(doc_length_list[i]+ave_length))*(word_count_in_collection[j]/total_words)
            temp = temp1 + temp2
            logp = logp+math.log(temp)
      
        p_for_doc_list.append(math.pow(math.e,logp))
        
                
    return p_for_doc_list

In [48]:
# for i in range(len(claims_10)):
    
#     claim_wordlist = claimProcess(claims_10[i]['claim'])
#     p_for_doc_list = DirichletSmoothing(claim_wordlist)
#     doc_id = getTop5(p_for_doc_list)

#     with open('q3_Dirichlet_smoothing.csv', "a", newline='') as config_csv:
#         writer = csv.writer(config_csv)
#         writer.writerow([f"The five most similar documents for Claim {claims_10[i]['id']} \n"])
#         for top_id in doc_id:
#             writer.writerow([str(top_id), str(wikipage[0][top_id])])
#         writer.writerow(f"\n")

In [49]:
# find the five most similar documents for ten claim and write them to csv
    
with open('q3.csv', "a", newline='') as config_csv:
    
    list_save = [' ']
    writer = csv.writer(config_csv)
    writer.writerow(list_save)
        
    list_save = ['Dirichlet_smoothing: ']
    writer = csv.writer(config_csv)
    writer.writerow(list_save)
    
    
    list_save = ['claim id', 'doc id_1', 'doc id_2', 'doc id_3', 'doc id_4', 'doc id_5']   
    writer = csv.writer(config_csv)
    writer.writerow(list_save)
    
for i in tqdm(range(0,len(claims_10))):  
    
    claim_wordlist = claimProcess(claims_10[i]['claim'])
    p_for_doc_list = DirichletSmoothing(claim_wordlist)
    doc_id = getTop5(p_for_doc_list)
    
    with open('q3.csv', "a", newline='') as config_csv:
        
        list_save = [claims_10[i]['id']]
        list_save.extend(wikipage[0][doc_id])
        
        writer = csv.writer(config_csv)
        writer.writerow(list_save)